In [1]:
import time
import pandas as pd
from zomatopy import Zomato
#get_categories
#get_city_ID
#get_city_name
#get_collections
#get_cuisines
#get_establishment_types
#get_nearby_restaurants
#get_restaurant
#restaurant_search

zconfig = {"user_key":"2dde89df168a9311ecd9f1459126144c"}

### READ DATA

In [2]:
zcities_df = pd.read_csv('../data/kin_nov2/zcities_cleaned.csv') # cleaned zomato cities
cstates_df = pd.read_csv('../data/kin_nov2/top_population_state.csv') # census top 10 migrant populations
cuisines_df = pd.read_csv('../data/kin_nov2/cuisine_ids.csv') # merged cuisines and populations

In [3]:
zcities_df.drop('Unnamed: 0', axis=1, inplace=True)
zcities_df.head(5)

,zcity_id,city_query,state_code
0,276,Dallas TX,TX
1,277,Houston TX,TX
2,278,Austin TX,TX
3,279,Seattle WA,WA
4,280,New York NY,NY


In [4]:
cstates_df.head(5)

,state,state_code,population1,population2,population3,population4,population5,population6,population7,population8,population9,population10
0,ALABAMA,AL,Mexico,Guatemala,Korea,India,China,Germany,Vietnam,Philippines,Honduras,Canada
1,ALASKA,AK,Philippines,Mexico,Korea,Thailand,Canada,Russia,Germany,Dominican Republic,"Oceania, nec",Laos
2,ARIZONA,AZ,Mexico,Canada,India,Philippines,China,Vietnam,Germany,Guatemala,Korea,Iraq
3,ARKANSAS,AR,Mexico,El Salvador,India,"Oceania, nec",Guatemala,Vietnam,China,Philippines,Laos,Germany
4,CALIFORNIA,CA,Mexico,Philippines,China,Vietnam,India,El Salvador,Korea,Guatemala,Iran,Taiwan


In [5]:
cuisines_df.head(5)

,population,cuisine_names,cuisine_id
0,Mexico,"Mexico, Taco, Tex-Mex","73,997,150"
1,Canada,Canada,381
2,India,India,148
3,China,"China, Sichuan","25,128"
4,Vietnam,Vietnam,99


### QUERY DATAFRAME

In [6]:
query_df = zcities_df.copy()
query_df = query_df.merge(cstates_df, on='state_code', how='left')
perc = query_df.population1.isna().sum()/len(query_df)
print(f'missing state merge percentage {perc}')

missing state merge percentage 0.0


In [7]:
query_df.head(5)

,zcity_id,city_query,state_code,state,population1,population2,population3,population4,population5,population6,population7,population8,population9,population10
0,276,Dallas TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
1,277,Houston TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
2,278,Austin TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
3,279,Seattle WA,WA,WASHINGTON,Mexico,India,Philippines,China,Vietnam,Korea,Canada,Ukraine,Russia,Germany
4,280,New York NY,NY,NEW YORK,Dominican Republic,China,Jamaica,Mexico,Ecuador,Guyana,India,Haiti,El Salvador,Colombia


In [8]:
query_df = pd.wide_to_long(query_df, stubnames='population',
                           i=['zcity_id', 'state_code'], j='top')
query_df = query_df.reset_index()
query_df.head(10)

,zcity_id,state_code,top,state,city_query,population
0,276,TX,1,TEXAS,Dallas TX,Mexico
1,276,TX,2,TEXAS,Dallas TX,India
2,276,TX,3,TEXAS,Dallas TX,El Salvador
3,276,TX,4,TEXAS,Dallas TX,Vietnam
4,276,TX,5,TEXAS,Dallas TX,Honduras
5,276,TX,6,TEXAS,Dallas TX,China
6,276,TX,7,TEXAS,Dallas TX,Philippines
7,276,TX,8,TEXAS,Dallas TX,Guatemala
8,276,TX,9,TEXAS,Dallas TX,Nigeria
9,276,TX,10,TEXAS,Dallas TX,Pakistan


In [9]:
query_df = query_df.reset_index()

In [10]:
query_df = query_df.merge(cuisines_df, on='population', how='left')
perc = query_df.cuisine_id.isna().sum()/len(query_df)
print(f'missing cuisine_id percentage {perc}')
check_missings = query_df[query_df.cuisine_id.isna()].population
check_missings = check_missings.unique()
query_df.head(10)

missing cuisine_id percentage 0.0


,index,zcity_id,state_code,top,state,city_query,population,cuisine_names,cuisine_id
0,0,276,TX,1,TEXAS,Dallas TX,Mexico,"Mexico, Taco, Tex-Mex","73,997,150"
1,1,276,TX,2,TEXAS,Dallas TX,India,India,148
2,2,276,TX,3,TEXAS,Dallas TX,El Salvador,El Salvador,601
3,3,276,TX,4,TEXAS,Dallas TX,Vietnam,Vietnam,99
4,4,276,TX,5,TEXAS,Dallas TX,Honduras,South America,972
5,5,276,TX,6,TEXAS,Dallas TX,China,"China, Sichuan","25,128"
6,5,276,TX,6,TEXAS,Dallas TX,China,"Hong Kong, Dim Sum","121, 411"
7,6,276,TX,7,TEXAS,Dallas TX,Philippines,Philippines,112
8,7,276,TX,8,TEXAS,Dallas TX,Guatemala,Guatemala,"972,136"
9,8,276,TX,9,TEXAS,Dallas TX,Nigeria,Nigeria,152


In [11]:
check_missings

array([], dtype=object)

### Zomato top populations/cuisines per city query

In [ ]:
# import requests
# import json
# from Levenshtein import distance as levenshtein_distance

# base_url = "https://developers.zomato.com/api/v2.1/"


# def clean_punctuation(my_str):
#     my_str = str(my_str).lower()
#     punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
#     for ele in my_str:  
#         if ele in punc:  
#             my_str = my_str.replace(ele, "")
#     return my_str


# class Zomato:
#     def __init__(self, config):
#         self.user_key = config["user_key"]


#     def get_categories(self):
#         """
#         Takes no input.
#         Returns a dictionary of IDs and their respective category names.
#         """
#         headers = {'Accept': 'application/json', 'user-key': self.user_key}
#         r = (requests.get(base_url + "categories", headers=headers).content).decode("utf-8")
#         a = json.loads(r)

#         self.is_key_invalid(a)
#         self.is_rate_exceeded(a)

#         categories = {}
#         for category in a['categories']:
#             categories.update({category['categories']['id'] : category['categories']['name']})

#         return categories


#     def get_city_ID(self, city_name):
#         """
#         Takes City Name as input.
#         Returns the ID for the city given as input.
#         """
#         city_name = clean_punctuation(city_name)
#         if all(x.isalpha() or x.isspace() for x in city_name) == False:
#             raise ValueError('InvalidCityName')
#         city_name = city_name.split(' ')
#         city_name = '%20'.join(city_name)
#         headers = {'Accept': 'application/json', 'user-key': self.user_key}
#         r = (requests.get(base_url + "cities?q=" + city_name, headers=headers).content).decode("utf-8")
#         a = json.loads(r)

#         self.is_key_invalid(a)
#         self.is_rate_exceeded(a)
#         self.json = a

#         if len(a['location_suggestions']) == 0:
#             raise Exception('invalid_city_name')
#         elif 'name' in a['location_suggestions'][0]:
#             city_query = city_name.replace('%20', ' ')

#             city_suggested = a['location_suggestions'][0]['name']
#             city_query = clean_punctuation(city_query)
#             city_suggested = clean_punctuation(city_suggested)
#             lev_distance = levenshtein_distance(city_suggested, city_query)
#             if lev_distance<5:
#                 return a['location_suggestions'][0]['id']
#             else:
#                 return -9999, city_suggested
            
#     def get_city_restaurant_count(self, city_ID="", cuisines="", limit=5):
#         """
#         Takes either query, latitude and longitude or cuisine as input.
#         Returns a list of Restaurant IDs.
#         """
#         #https://developers.zomato.com/api/v2.1/search?entity_id=280&entity_type=city&cuisines=73%2C%2025
#         cuisines = cuisines.strip()
#         cuisines = "%2C".join(cuisines.split(","))
        
#         self.is_valid_city_id(city_ID)
        
#         if str(limit).isalpha() == True:
#             raise ValueError('LimitNotInteger')
        
#         headers = {'Accept': 'application/json', 'user-key': self.user_key}
#         query_str = base_url + "search?" + "&entity_id" + str(city_ID) + "entity_type=city"\
#                                 + "&count=" + str(limit) + "&cuisines=" + str(cuisines)
        
#         r = (requests.get(query_str, headers=headers).content).decode("utf-8")
#         a = json.loads(r)
        
#         restaurant_count = a['results_found']
#         return restaurant_count

#     def get_city_name(self, city_ID):
#         """
#         Takes City ID as input.
#         Returns the name of the city ID given as input.
#         """
#         self.is_valid_city_id(city_ID)

#         headers = {'Accept': 'application/json', 'user-key': self.user_key}
#         r = (requests.get(base_url + "cities?city_ids=" + str(city_ID), headers=headers).content).decode("utf-8")
#         a = json.loads(r)

#         self.is_key_invalid(a)
#         self.is_rate_exceeded(a)

#         if a['location_suggestions'][0]['country_name'] == "":
#             raise ValueError('InvalidCityId')
#         else:
#             temp_city_ID = a['location_suggestions'][0]['id']
#             if temp_city_ID == str(city_ID):
#                 return a['location_suggestions'][0]['name']



#     def get_collections(self, city_ID, limit=None):
#         """
#         Takes City ID as input. limit parameter is optional.
#         Returns dictionary of Zomato restaurant collections in a city and their respective URLs.
#         """
#         self.is_valid_city_id(city_ID)

#         headers = {'Accept': 'application/json', 'user-key': self.user_key}
#         if limit == None:
#             r = (requests.get(base_url + "collections?city_id=" + str(city_ID), headers=headers).content).decode("utf-8")
#         else:
#             if str(limit).isalpha() == True:
#                 raise ValueError('LimitNotInteger')
#             else:
#                 r = (requests.get(base_url + "collections?city_id=" + str(city_ID) + "&count=" + str(limit), headers=headers).content).decode("utf-8")
#         a = json.loads(r)

#         self.is_key_invalid(a)
#         self.is_rate_exceeded(a)

#         collections = {}
#         for collection in a['collections']:
#             collections.update({collection['collection']['title'] : collection['collection']['url']})

#         return collections



#     def get_cuisines(self, city_ID):
#         """
#         Takes City ID as input.
#         Returns a sorted dictionary of all cuisine IDs and their respective cuisine names.
#         """
#         self.is_valid_city_id(city_ID)

#         headers = {'Accept': 'application/json', 'user-key': self.user_key}
#         r = (requests.get(base_url + "cuisines?city_id=" + str(city_ID), headers=headers).content).decode("utf-8")
#         a = json.loads(r)

#         self.is_key_invalid(a)
#         self.is_rate_exceeded(a)

#         if len(a['cuisines']) == 0:
#             raise ValueError('InvalidCityId')
#         temp_cuisines = {}
#         cuisines = {}
#         for cuisine in a['cuisines']:
#             temp_cuisines.update({cuisine['cuisine']['cuisine_id'] : cuisine['cuisine']['cuisine_name']})

#         for cuisine in sorted(temp_cuisines):
#             cuisines.update({cuisine : temp_cuisines[cuisine]})

#         return cuisines



#     def get_establishment_types(self, city_ID):
#         """
#         Takes City ID as input.
#         Returns a sorted dictionary of all establishment type IDs and their respective establishment type names.
#         """
#         self.is_valid_city_id(city_ID)

#         headers = {'Accept': 'application/json', 'user-key': self.user_key}
#         r = (requests.get(base_url + "establishments?city_id=" + str(city_ID), headers=headers).content).decode("utf-8")
#         a = json.loads(r)

#         self.is_key_invalid(a)
#         self.is_rate_exceeded(a)

#         temp_establishment_types = {}
#         establishment_types = {}
#         if 'establishments' in a:
#             for establishment_type in a['establishments']:
#                 temp_establishment_types.update({establishment_type['establishment']['id'] : establishment_type['establishment']['name']})

#             for establishment_type in sorted(temp_establishment_types):
#                 establishment_types.update({establishment_type : temp_establishment_types[establishment_type]})

#             return establishment_types
#         else:
#             raise ValueError('InvalidCityId')



#     def get_nearby_restaurants(self, latitude, longitude):
#         """
#         Takes the latitude and longitude as inputs.
#         Returns a dictionary of Restaurant IDs and their corresponding Zomato URLs.
#         """
#         try:
#             float(latitude)
#             float(longitude)
#         except ValueError:
#             raise ValueError('InvalidLatitudeOrLongitude')

#         headers = {'Accept': 'application/json', 'user-key': self.user_key}
#         r = (requests.get(base_url + "geocode?lat=" + str(latitude) + "&lon=" + str(longitude), headers=headers).content).decode("utf-8")
#         a = json.loads(r)

#         nearby_restaurants = {}
#         for nearby_restaurant in a['nearby_restaurants']:
#             nearby_restaurants.update({nearby_restaurant['restaurant']['id'] : nearby_restaurant['restaurant']['url']})

#         return nearby_restaurants



#     def get_restaurant(self, restaurant_ID):
#         """
#         Takes Restaurant ID as input.
#         Returns a dictionary of restaurant details.
#         """
#         self.is_valid_restaurant_id(restaurant_ID)

#         headers = {'Accept': 'application/json', 'user-key': self.user_key}
#         r = (requests.get(base_url + "restaurant?res_id=" + str(restaurant_ID), headers=headers).content).decode("utf-8")
#         a = json.loads(r)

#         if 'code' in a:
#             if a['code'] == 404:
#                 raise('InvalidRestaurantId')

#         restaurant_details = {}
#         restaurant_details.update({"name" : a['name']})
#         restaurant_details.update({"url" : a['url']})
#         restaurant_details.update({"location" : a['location']['address']})
#         restaurant_details.update({"city" : a['location']['city']})
#         restaurant_details.update({"city_ID" : a['location']['city_id']})
#         restaurant_details.update({"user_rating" : a['user_rating']['aggregate_rating']})

#         restaurant_details = DotDict(restaurant_details)
#         return restaurant_details


#     def restaurant_search(self, query="", latitude="", longitude="", cuisines="", limit=5):
#         """
#         Takes either query, latitude and longitude or cuisine as input.
#         Returns a list of Restaurant IDs.
#         """
#         cuisines = "%2C".join(cuisines.split(","))
#         if str(limit).isalpha() == True:
#             raise ValueError('LimitNotInteger')
#         headers = {'Accept': 'application/json', 'user-key': self.user_key}
#         r = (requests.get(base_url + "search?q=" + str(query) + "&count=" + str(limit) + "&lat=" + str(latitude) + "&lon=" + str(longitude) + "&cuisines=" + str(cuisines), headers=headers).content).decode("utf-8")
#         a = json.loads(r)

#         restaurants = []

#         if a['results_found'] == 0:
#             return []
#         else:
#             for restaurant in a['restaurants']:
#                 restaurants.append(restaurant['restaurant']['id'])

#         return restaurants




#     def is_valid_restaurant_id(self, restaurant_ID):
#         """
#         Checks if the Restaurant ID is valid or invalid.
#         If invalid, throws a InvalidRestaurantId Exception.
#         """
#         restaurant_ID = str(restaurant_ID)
#         if restaurant_ID.isnumeric() == False:
#             raise ValueError('InvalidRestaurantId')



#     def is_valid_city_id(self, city_ID):
#         """
#         Checks if the City ID is valid or invalid.
#         If invalid, throws a InvalidCityId Exception.
#         """
#         city_ID = str(city_ID)
#         if city_ID.isnumeric() == False:
#             raise ValueError('InvalidCityId')



#     def is_key_invalid(self, a):
#         """
#         Checks if the API key provided is valid or invalid.
#         If invalid, throws a InvalidKey Exception.
#         """
#         if 'code' in a:
#             if a['code'] == 403:
#                 raise ValueError('InvalidKey')



#     def is_rate_exceeded(self, a):
#         """
#         Checks if the request limit for the API key is exceeded or not.
#         If exceeded, throws a ApiLimitExceeded Exception.
#         """
#         if 'code' in a:
#             if a['code'] == 440:
#                 raise Exception('ApiLimitExceeded')



# class DotDict(dict):
#     """
#     Dot notation access to dictionary attributes
#     """

#     __getattr__ = dict.get
#     __setattr__ = dict.__setitem__
#     __delattr__ = dict.__delitem__

In [ ]:
zomato = Zomato(config=zconfig)

In [ ]:
zomato.get_city_ID(city_name='Chicago')

In [ ]:
# def get_city_restaurant_count(self, city_ID="", cuisines="", limit=5):
#     """
#     Takes either query, latitude and longitude or cuisine as input.
#     Returns a list of Restaurant IDs.
#     """
#     #https://developers.zomato.com/api/v2.1/search?entity_id=280&entity_type=city&cuisines=73%2C%2025

ver = zomato.get_city_restaurant_count(city_ID=280, cuisines="73,25")

In [20]:
def zomato_city_restaurant_count_query(query_df, zconfig):
    zomato = Zomato(zconfig)
    
    zcity_id_list = []
    cuisine_id_list = []
    json_list = []
    count_list = []
    for i, row in query_df.iterrows():
        print(f'city: {row.zcity_id} cuisines: {row.cuisine_id}')
        
        # Security saves of Zomato calls
        if i>0 and (i % 10 == 0):
            time.sleep(5)
            response_df = pd.DataFrame.from_dict({'zcity_id': zcity_ids, 'json': jsons, 
                                                 'city_query': city_queries})
            response_df.to_pickle('../results/zcities_df.pkl')
        
        try:
            zcity_id = zomato.get_city_ID(city_name=row.city_query)

            city_queries += [row.city_query]
            zcity_ids += [zcity_id]
            jsons += [zomato.json]
        except:
            print(f"failed city query {row.city_query}")
            pass

        zcity_id_list += [row.zcity_id]
        cuisine_id_list += [row.cuisine_id]
        json_list += ['hola']
        count_list += [1]
    
    # Final save
    response_df = pd.DataFrame.from_dict({'zcity_id': zcity_id_list, 
                                                  'cuisines': cuisine_id_list,
                                                  'json': json_list,
                                                  'counts': count_list})
    city_restaurants_df.to_pickle('../results/city_restaurant_count.pkl')
    return city_restaurants_df

city_restaurants_df = zomato_city_restaurant_count_query(query_df, zconfig)

city: 276 cuisines: 73,997,150
city: 276 cuisines: 148
city: 276 cuisines: 601
city: 276 cuisines: 99
city: 276 cuisines: 972
city: 276 cuisines: 25,128
city: 276 cuisines: 121, 411
city: 276 cuisines: 112
city: 276 cuisines: 972,136
city: 276 cuisines: 152
city: 276 cuisines: 139
city: 277 cuisines: 73,997,150
city: 277 cuisines: 148
city: 277 cuisines: 601
city: 277 cuisines: 99
city: 277 cuisines: 972
city: 277 cuisines: 25,128
city: 277 cuisines: 121, 411
city: 277 cuisines: 112
city: 277 cuisines: 972,136
city: 277 cuisines: 152
city: 277 cuisines: 139
city: 278 cuisines: 73,997,150
city: 278 cuisines: 148
city: 278 cuisines: 601
city: 278 cuisines: 99
city: 278 cuisines: 972
city: 278 cuisines: 25,128
city: 278 cuisines: 121, 411
city: 278 cuisines: 112
city: 278 cuisines: 972,136
city: 278 cuisines: 152
city: 278 cuisines: 139
city: 279 cuisines: 73,997,150
city: 279 cuisines: 148
city: 279 cuisines: 112
city: 279 cuisines: 25,128
city: 279 cuisines: 121, 411
city: 279 cuisines:

city: 733 cuisines: 152
city: 742 cuisines: 73,997,150
city: 742 cuisines: 148
city: 742 cuisines: 153
city: 742 cuisines: 25,128
city: 742 cuisines: 121, 411
city: 742 cuisines: 972,136
city: 742 cuisines: 112
city: 742 cuisines: 134
city: 742 cuisines: 99
city: 742 cuisines: 67
city: 742 cuisines: 651
city: 745 cuisines: 73,997,150
city: 745 cuisines: 148
city: 745 cuisines: 153
city: 745 cuisines: 25,128
city: 745 cuisines: 121, 411
city: 745 cuisines: 972,136
city: 745 cuisines: 112
city: 745 cuisines: 134
city: 745 cuisines: 99
city: 745 cuisines: 67
city: 745 cuisines: 651
city: 751 cuisines: 73,997,150
city: 751 cuisines: 148
city: 751 cuisines: 153
city: 751 cuisines: 25,128
city: 751 cuisines: 121, 411
city: 751 cuisines: 972,136
city: 751 cuisines: 112
city: 751 cuisines: 134
city: 751 cuisines: 99
city: 751 cuisines: 67
city: 751 cuisines: 651
city: 757 cuisines: 73,997,150
city: 757 cuisines: 972
city: 757 cuisines: 99
city: 757 cuisines: 972,136
city: 757 cuisines: 148
cit

city: 1189 cuisines: 99
city: 1189 cuisines: 972
city: 1189 cuisines: 25,128
city: 1189 cuisines: 121, 411
city: 1189 cuisines: 112
city: 1189 cuisines: 972,136
city: 1189 cuisines: 152
city: 1189 cuisines: 139
city: 1192 cuisines: 73,997,150
city: 1192 cuisines: 148
city: 1192 cuisines: 601
city: 1192 cuisines: 99
city: 1192 cuisines: 972
city: 1192 cuisines: 25,128
city: 1192 cuisines: 121, 411
city: 1192 cuisines: 112
city: 1192 cuisines: 972,136
city: 1192 cuisines: 152
city: 1192 cuisines: 139
city: 1195 cuisines: 73,997,150
city: 1195 cuisines: 148
city: 1195 cuisines: 601
city: 1195 cuisines: 99
city: 1195 cuisines: 972
city: 1195 cuisines: 25,128
city: 1195 cuisines: 121, 411
city: 1195 cuisines: 112
city: 1195 cuisines: 972,136
city: 1195 cuisines: 152
city: 1195 cuisines: 139
city: 1198 cuisines: 73,997,150
city: 1198 cuisines: 148
city: 1198 cuisines: 601
city: 1198 cuisines: 99
city: 1198 cuisines: 972
city: 1198 cuisines: 25,128
city: 1198 cuisines: 121, 411
city: 1198 cui

city: 7955 cuisines: 73,997,150
city: 7955 cuisines: 112
city: 7955 cuisines: 25,128
city: 7955 cuisines: 121, 411
city: 7955 cuisines: 99
city: 7955 cuisines: 148
city: 7955 cuisines: 601
city: 7955 cuisines: 67
city: 7955 cuisines: 972,136
city: 7955 cuisines: 140
city: 7955 cuisines: 190
city: 8181 cuisines: 73,997,150
city: 8181 cuisines: 99
city: 8181 cuisines: 148
city: 8181 cuisines: 972,136
city: 8181 cuisines: 25,128
city: 8181 cuisines: 121, 411
city: 8181 cuisines: 112
city: 8181 cuisines: 67
city: 8181 cuisines: 134
city: 8181 cuisines: 972
city: 8181 cuisines: 381
city: 8279 cuisines: 73,997,150
city: 8279 cuisines: 99
city: 8279 cuisines: 148
city: 8279 cuisines: 972,136
city: 8279 cuisines: 25,128
city: 8279 cuisines: 121, 411
city: 8279 cuisines: 112
city: 8279 cuisines: 67
city: 8279 cuisines: 134
city: 8279 cuisines: 972
city: 8279 cuisines: 381
city: 8340 cuisines: 73,997,150
city: 8340 cuisines: 99
city: 8340 cuisines: 25,128
city: 8340 cuisines: 121, 411
city: 8340

city: 10853 cuisines: 25,128
city: 10853 cuisines: 121, 411
city: 10853 cuisines: 99
city: 10853 cuisines: 148
city: 10853 cuisines: 601
city: 10853 cuisines: 67
city: 10853 cuisines: 972,136
city: 10853 cuisines: 140
city: 10853 cuisines: 190
city: 10854 cuisines: 73,997,150
city: 10854 cuisines: 112
city: 10854 cuisines: 25,128
city: 10854 cuisines: 121, 411
city: 10854 cuisines: 99
city: 10854 cuisines: 148
city: 10854 cuisines: 601
city: 10854 cuisines: 67
city: 10854 cuisines: 972,136
city: 10854 cuisines: 140
city: 10854 cuisines: 190
city: 10874 cuisines: 73,997,150
city: 10874 cuisines: 112
city: 10874 cuisines: 25,128
city: 10874 cuisines: 121, 411
city: 10874 cuisines: 99
city: 10874 cuisines: 148
city: 10874 cuisines: 601
city: 10874 cuisines: 67
city: 10874 cuisines: 972,136
city: 10874 cuisines: 140
city: 10874 cuisines: 190
city: 10877 cuisines: 73,997,150
city: 10877 cuisines: 112
city: 10877 cuisines: 25,128
city: 10877 cuisines: 121, 411
city: 10877 cuisines: 99
city: 

In [ ]:
restaurant_count

In [14]:
query_df

,index,zcity_id,state_code,top,state,city_query,population,cuisine_names,cuisine_id
0,0,276,TX,1,TEXAS,Dallas TX,Mexico,"Mexico, Taco, Tex-Mex","73,997,150"
1,1,276,TX,2,TEXAS,Dallas TX,India,India,148
2,2,276,TX,3,TEXAS,Dallas TX,El Salvador,El Salvador,601
3,3,276,TX,4,TEXAS,Dallas TX,Vietnam,Vietnam,99
4,4,276,TX,5,TEXAS,Dallas TX,Honduras,South America,972
...,...,...,...,...,...,...,...,...,...
4392,4005,11268,CA,6,CALIFORNIA,Montebello CA,El Salvador,El Salvador,601
4393,4006,11268,CA,7,CALIFORNIA,Montebello CA,Korea,Korea,67
4394,4007,11268,CA,8,CALIFORNIA,Montebello CA,Guatemala,Guatemala,"972,136"
4395,4008,11268,CA,9,CALIFORNIA,Montebello CA,Iran,Iran,140


In [21]:
city_restaurants_df

,zcity_id,cuisines,json,counts
0,276,"73,997,150",hola,1
1,276,148,hola,1
2,276,601,hola,1
3,276,99,hola,1
4,276,972,hola,1
...,...,...,...,...
4392,11268,601,hola,1
4393,11268,67,hola,1
4394,11268,"972,136",hola,1
4395,11268,140,hola,1
